In [216]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from scipy.spatial import ConvexHull
scaler=StandardScaler()

,isTrueTrack,pdg,completeness,purity,x,z,length,sliding,eps,pc2Var,nHits,RSS,RSS2,sliding_modified
0,1,2212,1.000000,0.909091,"[1.72973107539, 1.64524385622, 1.51650142701, ...","[-1.69030850946, -1.60579308398, -1.5212776585...",4.785806,0.005515,0.133,0.000208,41,0.015505,0.000631,0.005525
1,1,2212,0.733333,1.000000,"[-1.47290088225, -1.33730088739, -1.0958667502...","[-1.60356745147, -1.33630620956, -1.0690449676...",4.449725,0.001786,0.365,0.000672,13,0.027864,0.001706,0.005464
2,0,11,0.892892,0.998911,"[1.6456069206, 1.63333808742, 1.61401467518, 1...","[-1.81425037168, -1.7974536461, -1.78065692051...",7.127999,0.157412,1.486,0.039021,247,0.227368,0.149946,0.111450
3,1,2212,0.989362,0.965398,"[-1.36891816532, -1.37278190977, -1.3722299462...","[-1.59732439026, -1.57583288679, -1.5543413833...",4.540850,0.004279,0.385,0.001711,136,0.051166,0.001112,0.004741
4,1,2212,0.875000,0.875000,"[-1.39488761287, -1.32920904814, -1.271740304,...","[-1.65144564769, -1.48630108292, -1.3211565181...",4.564701,0.003981,0.229,0.002311,21,0.055676,0.002714,0.004950


In [132]:
df = pd.read_csv('../ROOT_DATA/big_data_nue_&_mu.csv', header=None, names = ['isTrueTrack', 'pdg', 'completeness', 'purity', 'x', 'z'])

In [133]:
for j in range(len(df)):
    print(j)
    df['x'][j] =[float(i) for i in df['x'][j].split()]
    df['z'][j] = [float(i) for i in df['z'][j].split()]
    df['x'][j] = scaler.fit_transform(np.array(df['x'][j]).reshape(-1,1)).reshape(1,-1)[0]
    df['z'][j] =  scaler.fit_transform(np.array(df['z'][j]).reshape(-1,1)).reshape(1,-1)[0]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704


5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343


6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986


8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627


...

12790
12791
12792
12793
12794
12795
12796
12797
12798
12799
12800
12801
12802
12803
12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
12939
12940
12941
12942
12943
12944
12945
12946
12947
12948
12949
12950
12951
12952
12953
12954
12955
1295

15522
15523
15524
15525
15526
15527
15528
15529
15530
15531
15532
15533
15534
15535
15536
15537
15538
15539
15540
15541
15542
15543
15544
15545
15546
15547
15548
15549
15550
15551
15552
15553
15554
15555
15556
15557
15558
15559
15560
15561
15562
15563
15564
15565
15566
15567
15568
15569
15570
15571
15572
15573
15574
15575
15576
15577
15578
15579
15580
15581
15582
15583
15584
15585
15586
15587
15588
15589
15590
15591
15592
15593
15594
15595
15596
15597
15598
15599
15600
15601
15602
15603
15604
15605
15606
15607
15608
15609
15610
15611
15612
15613
15614
15615
15616
15617
15618
15619
15620
15621
15622
15623
15624
15625
15626
15627
15628
15629
15630
15631
15632
15633
15634
15635
15636
15637
15638
15639
15640
15641
15642
15643
15644
15645
15646
15647
15648
15649
15650
15651
15652
15653
15654
15655
15656
15657
15658
15659
15660
15661
15662
15663
15664
15665
15666
15667
15668
15669
15670
15671
15672
15673
15674
15675
15676
15677
15678
15679
15680
15681
15682
15683
15684
15685
15686
15687
1568

16896
16897
16898
16899
16900
16901
16902
16903
16904
16905
16906
16907
16908
16909
16910
16911
16912
16913
16914
16915
16916
16917
16918
16919
16920
16921
16922
16923
16924
16925
16926
16927
16928
16929
16930
16931
16932
16933
16934
16935
16936
16937
16938
16939
16940
16941
16942
16943
16944
16945
16946
16947
16948
16949
16950
16951
16952
16953
16954
16955
16956
16957
16958
16959
16960
16961
16962
16963
16964
16965
16966
16967
16968
16969
16970
16971
16972
16973
16974
16975
16976
16977
16978
16979
16980
16981
16982
16983
16984
16985
16986
16987
16988
16989
16990
16991
16992
16993
16994
16995
16996
16997
16998
16999
17000
17001
17002
17003
17004
17005
17006
17007
17008
17009
17010
17011
17012
17013
17014
17015
17016
17017
17018
17019
17020
17021
17022
17023
17024
17025
17026
17027
17028
17029
17030
17031
17032
17033
17034
17035
17036
17037
17038
17039
17040
17041
17042
17043
17044
17045
17046
17047
17048
17049
17050
17051
17052
17053
17054
17055
17056
17057
17058
17059
17060
17061
1706

18262
18263
18264
18265
18266
18267
18268
18269
18270
18271
18272
18273
18274
18275
18276
18277
18278
18279
18280
18281
18282
18283
18284
18285
18286
18287
18288
18289
18290
18291
18292
18293
18294
18295
18296
18297
18298
18299
18300
18301
18302
18303
18304
18305
18306
18307
18308
18309
18310
18311
18312
18313
18314
18315
18316
18317
18318
18319
18320
18321
18322
18323
18324
18325
18326
18327
18328
18329
18330
18331
18332
18333
18334
18335
18336
18337
18338
18339
18340
18341
18342
18343
18344
18345
18346
18347
18348
18349
18350
18351
18352
18353
18354
18355
18356
18357
18358
18359
18360
18361
18362
18363
18364
18365
18366
18367
18368
18369
18370
18371
18372
18373
18374
18375
18376
18377
18378
18379
18380
18381
18382
18383
18384
18385
18386
18387
18388
18389
18390
18391
18392
18393
18394
18395
18396
18397
18398
18399
18400
18401
18402
18403
18404
18405
18406
18407
18408
18409
18410
18411
18412
18413
18414
18415
18416
18417
18418
18419
18420
18421
18422
18423
18424
18425
18426
18427
1842

19630
19631
19632
19633
19634
19635
19636
19637
19638
19639
19640
19641
19642
19643
19644
19645
19646
19647
19648
19649
19650
19651
19652
19653
19654
19655
19656
19657
19658
19659
19660
19661
19662
19663
19664
19665
19666
19667
19668
19669
19670
19671
19672
19673
19674
19675
19676
19677
19678
19679
19680
19681
19682
19683
19684
19685
19686
19687
19688
19689
19690
19691
19692
19693
19694
19695
19696
19697
19698
19699
19700
19701
19702
19703
19704
19705
19706
19707
19708
19709
19710
19711
19712
19713
19714
19715
19716
19717
19718
19719
19720
19721
19722
19723
19724
19725
19726
19727
19728
19729
19730
19731
19732
19733
19734
19735
19736
19737
19738
19739
19740
19741
19742
19743
19744
19745
19746
19747
19748
19749
19750
19751
19752
19753
19754
19755
19756
19757
19758
19759
19760
19761
19762
19763
19764
19765
19766
19767
19768
19769
19770
19771
19772
19773
19774
19775
19776
19777
19778
19779
19780
19781
19782
19783
19784
19785
19786
19787
19788
19789
19790
19791
19792
19793
19794
19795
1979

In [13]:
df.head()

,isTrueTrack,pdg,completeness,purity,x,z
0,1,2212,1.000000,0.909091,"[1.72973107539, 1.64524385622, 1.51650142701, ...","[-1.69030850946, -1.60579308398, -1.5212776585..."
1,1,2212,0.733333,1.000000,"[-1.47290088225, -1.33730088739, -1.0958667502...","[-1.60356745147, -1.33630620956, -1.0690449676..."
2,0,11,0.892892,0.998911,"[1.6456069206, 1.63333808742, 1.61401467518, 1...","[-1.81425037168, -1.7974536461, -1.78065692051..."
3,1,2212,0.989362,0.965398,"[-1.36891816532, -1.37278190977, -1.3722299462...","[-1.59732439026, -1.57583288679, -1.5543413833..."
4,1,2212,0.875000,0.875000,"[-1.39488761287, -1.32920904814, -1.271740304,...","[-1.65144564769, -1.48630108292, -1.3211565181..."


In [65]:
def RSS(df):
    rssList = []
    for i in range(len(df)):
        perp_dist=0
        length=len(df.z[i])
        reg = LinearRegression().fit(np.array(df.x[i]).reshape(-1,1), df.z[i])
        y_pred = reg.predict(np.array(df.x[i]).reshape(-1,1))
        for j in range(length):
            perp_dist+=abs(-reg.coef_[0]*df.x[i][j] + df.z[i][j] -reg.intercept_)/(reg.coef_[0]**2 + 1)**0.5
        #rssList.append(mean_squared_error(df.z[i], y_pred))
        rssList.append(perp_dist/length)
    return rssList

In [66]:
def RSS2(df):
    rssList = []
    for i in range(len(df)):
        x = np.array(df.x[i]).reshape(-1,1)
        x2 = np.multiply(np.array(df.x[i]), np.array(df.x[i])).reshape(-1,1)
        xx = [[x[i][0], x2[i][0]] for i in range(len(x))]
        reg = LinearRegression().fit(xx, df.z[i])
        y_pred = reg.predict(xx)
        rssList.append(mean_squared_error(df.z[i], y_pred))
    return rssList

In [67]:
def pc2Var(df):
    pc2VarList = []
    scaler = StandardScaler()
    df2 = df.copy(deep=True)
    for i in range(len(df)):
        #print(i)
        df2.x[i] = scaler.fit_transform(np.array(df2.x[i]).reshape(-1,1))
        #df.z[i] = scaler.fit_transform(np.array(df.z[i]).reshape(-1,1))
        data = [[np.array(df2.x[i])[j], np.array(df2.z[i])[j]] for j in range(len(df2.x[i]))]
        data = np.array(data)
        data = data.reshape(data.shape[0],data.shape[1])
        print(data.shape)
        pca = PCA(n_components=1)
        pca.fit_transform(data)
        pc2VarList.append(1- pca.explained_variance_ratio_)
    return pc2VarList

In [217]:
def convexHullArea(df):
    areaList=[]
    for i in range(len(df)):
        data = [[np.array(df.x[i])[j], np.array(df.z[i])[j]] for j in range(len(df.x[i]))]
        #print(data,i)
        try:
            hull = ConvexHull(data)
            areaList.append(hull.area)
        except:
            areaList.append(0)
        print(i, hull.area)
    return areaList

In [219]:
area = convexHullArea(df)

0 9.579459152585578
1 8.9166420135929
2 15.046738384364446
3 9.154247191697129
4 9.176259273293686
5 15.307257186637951
6 10.396432823256102
7 18.23614820940696
8 10.112641753548667
9 9.539037952521221
10 20.255864153533054
11 13.754952145423324
12 9.624976003050786
13 9.496917709004292
14 18.920223148102544
15 9.232545912257104
16 8.429437246529515
17 9.676754020796032
18 11.011329659414313
19 10.337761606384367
20 9.556911974677213
21 9.140654107667109
22 10.285491613361767
23 12.22734509175586
24 9.62878221482449
25 8.868131671356124
26 23.88908329862134
27 28.318552669784005
28 12.65519279072795
29 12.26367508079214
30 9.10350498918939
31 9.64461717060594
32 9.647211102616012
33 10.180703771837454
34 9.17876253440586
35 16.918884122988082
36 8.014744345614993
37 6.909294536130703
38 12.803027717499269
39 9.770174796537175
40 19.247564580366014
41 11.382826672751694
42 7.868066096243989
43 7.9633948333306535
44 15.541443949274816
45 12.085369454076181
46 10.105385380883332
47 8.6961

399 7.1773499634146125
400 14.192485473801627
401 17.156047860350515
402 6.9278863737592795
403 9.965929052411031
404 19.562780645782286
405 20.04588339238394
406 9.922186777006834
407 14.058680256404823
408 9.530192517846892
409 8.435205903179956
410 14.635833504808264
411 10.75671224698379
412 14.550576130311134
413 10.312110967668346
414 18.197817708474815
415 15.531488481367555
416 10.758100332956698
417 7.163592123161376
418 23.900527234937712
419 9.473666684692184
420 11.258020183107533
421 10.336892636000236
422 9.469387771257072
423 13.325219283198276
424 16.633131074343073
425 13.810500273997036
426 11.87954409880291
427 7.929450741404776
428 9.397855388266006
429 20.813196744715075
430 13.476055987901933
431 10.133235409374116
432 8.81785216934068
433 7.327162037782816
434 24.37036030287884
435 21.280221037843532
436 12.341633627582823
437 14.703187671888436
438 13.615556153488917
439 8.182096304187862
440 17.461466988884403
441 9.375369913961421
442 14.20422929989168
443 10.

782 9.434577998278368
783 17.1523124666147
784 16.831552210107887
785 9.408562903924729
786 14.369499315790284
787 9.064974499801691
788 7.886228641661896
789 12.842360996821762
790 12.562784055443274
791 9.010434720447709
792 9.243732027558524
793 11.39083317677089
794 14.764199142623735
795 9.830386473438388
796 13.566405579675356
797 9.712126989882767
798 23.506542141630558
799 9.645388060112666
800 9.777755418408047
801 9.991341766133484
802 14.02416640612773
803 10.015599480068401
804 9.700225650628907
805 17.851738966316184
806 9.703440827247611
807 11.496775700008293
808 14.934913030031126
809 9.506150477128665
810 9.958829704258774
811 9.92612367431827
812 14.440404644958821
813 9.294646220520221
814 14.71418718547291
815 11.434913201301407
816 9.644157654161402
817 9.132495055191454
818 7.91694728823621
819 8.041928689699898
820 18.31117879475326
821 11.126284887342628
822 15.88622863137342
823 9.825009994392584
824 9.59873249196893
825 19.84381083565607
826 9.714674954616601


1152 19.634599191923183
1153 22.112855424293112
1154 11.102323809526355
1155 14.42854901201981
1156 6.897705965810899
1157 9.640784785826003
1158 6.90649335189526
1159 18.029301910003497
1160 14.684437900443811
1161 8.982196473601281
1162 14.701729007797162
1163 14.41023103148379
1164 10.001492361213412
1165 8.626286527906808
1166 8.823318375712889
1167 11.015783410970924
1168 10.822455440037636
1169 9.481680847502957
1170 6.927365301065555
1171 12.582261421554206
1172 9.061724253111256
1173 15.87678534240503
1174 15.14794241143996
1175 13.505801743947565
1176 9.703257079323445
1177 8.248615894565928
1178 8.12700501766093
1179 16.490241013575996
1180 9.11712149858736
1181 20.691596227316296
1182 9.232855229663226
1183 9.633610670537657
1184 9.269095540780953
1185 9.070586665067484
1186 13.951037963753938
1187 9.39826614485899
1188 8.450204846233499
1189 8.574058757876951
1190 9.616909279706366
1191 12.08813523146557
1192 13.01888184558075
1193 17.07626874106456
1194 9.167993550971037
1

1519 10.122150087547274
1520 12.13955774350788
1521 10.388058424408248
1522 7.875090265218287
1523 13.3710765496664
1524 8.405687163626297
1525 7.998400904228432
1526 14.559337064659564
1527 13.782450105258638
1528 13.453627718857332
1529 9.564808513668567
1530 9.36296961474125
1531 9.911634688987139
1532 17.43619647612792
1533 10.367691532217274
1534 9.66401228576711
1535 16.310268071680497
1536 10.266748328143162
1537 14.29840598155763
1538 10.118181347909943
1539 6.897764628389336
1540 15.319171415468412
1541 13.586551081733846
1542 12.998837082099783
1543 9.26976771924797
1544 17.395558530533336
1545 17.35301637002801
1546 13.74820557758578
1547 8.40555410422697
1548 7.874586059629296
1549 9.806388178657025
1550 9.292667821430612
1551 9.707062691833379
1552 10.383150354023142
1553 18.250603556083345
1554 10.20856753959676
1555 11.722142824785914
1556 8.90134644111837
1557 13.730869982882385
1558 11.941560811182802
1559 15.220314572445954
1560 10.598590548119018
1561 9.2420148166255

...

18734 9.456588754628125
18735 9.661830442187197
18736 9.599125756603598
18737 6.989659362354692
18738 11.58950324573357
18739 9.471984439973278
18740 9.995359207548923
18741 10.497930601456721
18742 9.654706041131115
18743 9.276775443903393
18744 9.34611211151773
18745 9.561445062076846
18746 9.710037248194539
18747 10.372138060338626
18748 8.218604593082791
18749 9.568759732726972
18750 10.12896668082358
18751 9.67204802465077
18752 8.76039131178228
18753 9.876009508710192
18754 10.648733717011858
18755 10.157246938922892
18756 9.941685372114058
18757 9.697717015122164
18758 9.614809988295216
18759 6.919228814177609
18760 9.714056666931631
18761 9.923781616895882
18762 10.090183082339232
18763 10.546824457496335
18764 9.280183257194727
18765 10.861701477382407
18766 7.313679588423035
18767 10.003464733790672
18768 9.816099440365015
18769 10.01635062760593
18770 7.5652513930296985
18771 9.374933169491444
18772 9.68787358249303
18773 12.840529981535585
18774 13.337970005490767
18775 9.6

19081 9.369022480020334
19082 9.729045127035418
19083 17.60086402472863
19084 11.110838226188493
19085 9.564343231404553
19086 9.826921503401197
19087 9.81053406707552
19088 9.196277212854929
19089 9.48144897372649
19090 9.419388882980194
19091 16.963590099059886
19092 9.66280677622186
19093 9.964320345972167
19094 6.900614581442248
19095 12.379340464648894
19096 9.845199407221145
19097 9.651933605736602
19098 9.081254072045162
19099 10.186756736808446
19100 10.215698533775123
19101 8.410337595285558
19102 9.73462350075343
19103 9.502932602509448
19104 10.133445838518602
19105 11.77204413489016
19106 10.662863774927079
19107 9.38248410252699
19108 9.680309173126231
19109 9.800242363439537
19110 8.831968361746569
19111 9.474771464910923
19112 10.772119777513732
19113 9.534142522812994
19114 9.892390465674058
19115 9.120478397136566
19116 11.340084546714062
19117 16.476284539397064
19118 12.767774055375254
19119 12.102189430318766
19120 9.188209387547822
19121 9.000694445172993
19122 10.

19453 10.517686923652779
19454 11.054944518689073
19455 9.710181304191075
19456 9.70560448975734
19457 10.125332592381433
19458 7.81350548275727
19459 6.931040695553831
19460 9.382509821705534
19461 9.606656525502848
19462 10.047790888650514
19463 8.241547081625232
19464 8.954997749416654
19465 8.343265507248836
19466 9.949629109296048
19467 9.491770625477933
19468 9.71631298644819
19469 9.544519495429293
19470 10.09711421037835
19471 7.318109103108927
19472 10.25117274498079
19473 9.875847213235645
19474 9.413348485765724
19475 11.458869525902701
19476 7.437101511173487
19477 9.171708529171005
19478 9.713758623056759
19479 9.462496629393431
19480 9.852973969581544
19481 9.75290510286432
19482 10.30428562714891
19483 9.814253617838702
19484 9.393140506277035
19485 17.847240544438684
19486 11.549658529761311
19487 6.9171202554461875
19488 11.055856708498363
19489 9.512592651116963
19490 9.01582081640269
19491 8.434609733200878
19492 10.079219822366147
19493 9.447678084309278
19494 10.79

19809 9.70513666623397
19810 9.504937685130079
19811 8.964980654760927
19812 9.783311429529462
19813 12.384190611083692
19814 9.991585485818955
19815 6.924969490952552
19816 9.602936370812161
19817 23.816083374740384
19818 10.017930791112652
19819 8.665449001570222
19820 11.813330895003402
19821 12.62826154921255
19822 9.41904044201246
19823 9.453716065817163
19824 9.620216486738602
19825 9.26935800463836
19826 9.186131774249306
19827 7.065970994227659
19828 10.067448407026468
19829 9.495131818759791
19830 9.362174997796268
19831 11.417767268263198
19832 7.966880572015656
19833 9.371285572683439
19834 10.366170477830972
19835 9.676903546396916
19836 9.599925686945156
19837 9.912066172841598
19838 13.038070070481593
19839 10.388897154675247
19840 11.161710041664463
19841 10.47414775565964
19842 9.528081010900186
19843 9.959816388129354
19844 9.362752293216083
19845 9.9354031903001
19846 9.78397727939086
19847 9.100388152338917
19848 9.35361211935232
19849 9.923022018603326
19850 11.0795

20152 9.132328089568354
20153 10.149172321647772
20154 9.979170923831111
20155 9.289359665050123
20156 9.696347279514363
20157 8.990169175178949
20158 7.957037741260731
20159 9.812733428246487
20160 9.410078610815885
20161 9.307156947642364
20162 9.482306214373828
20163 9.624976540638936
20164 7.925087716692389
20165 7.222794305881292
20166 8.854720597162407
20167 8.805629518301417
20168 9.92837633078125
20169 9.887208856528751
20170 11.763682911273195
20171 9.685896708281795
20172 9.749777442839386
20173 12.14644666747968
20174 9.355313876554494
20175 11.340315679985787
20176 9.186136748066996
20177 8.528167645149153
20178 8.580425795261664
20179 9.963688540934989
20180 8.389194246215403
20181 7.947840478565052
20182 7.635217602542425
20183 9.2249876443672
20184 10.263285834442991
20185 10.04000708336812
20186 12.77717171614322
20187 11.084306211788391
20188 10.006258563052368
20189 8.780602140187735
20190 11.835492514115826
20191 12.096124025595204
20192 10.168329049710795
20193 8.20

In [150]:
def length(df):
    lengthList = []
    scaler = StandardScaler()
    df2 = df.copy(deep=True)
    for i in range(len(df)):
        print(i)
        df2.x[i] = scaler.fit_transform(np.array(df2.x[i]).reshape(-1,1))
        #df.z[i] = scaler.fit_transform(np.array(df.z[i]).reshape(-1,1))
        data = [[np.array(df2.x[i])[j], np.array(df2.z[i])[j]] for j in range(len(df2.x[i]))]
        data = np.array(data)
        data = data.reshape(data.shape[0],data.shape[1])
        #print(data.shape)
        pca = PCA(n_components=1)
        new_data = pca.fit_transform(data)
        length = float(max(new_data)- min(new_data))
        lengthList.append(length)
        print(length)
    return lengthList

In [152]:
length = length(df)

0
4.785806307892756
1
4.449725121211145
2
7.127998917925451
3
4.5408501405537205
4
4.564701240505426
5
4.864244023641169
6
3.3475940347154385
7
7.973702067337388
8
5.04731009213215
9
4.767060416152019
10
9.953465771505023
11
6.852609776123681
12
3.8282075698380638
13
4.452211846656746
14
9.198900559714993
15
3.974864692719663
16
2.782389319177165
17
4.800107590407878
18
5.02939636639263
19
3.366269569712446
20
4.763033410964082
21
4.494114273431781
22
4.50657159663165
23
5.764342843215005
24
4.813403843184734
25
4.335505848670149
26
11.39958678866072
27
7.151701005176447
28
4.510915124946999
29
4.516349846053269
30
4.551078830784011
31
4.813409001025543
32
4.810545485633396
33
4.468146269065176
34
4.429993514683052
35
6.977078053053932
36
3.7828289709754346
37
3.416872006581996
38
6.269285544159993
39
4.838438391157913
40
8.8396666395959
41
5.078917751024818
42
3.6674811653793795
43
3.6921145734760676
44
6.467995887648417
45
3.856220378814784
46
3.9275525533836024
47
4.295732474754142


386
6.216041906078516
387
5.586597400583932
388
3.987483354851652
389
4.435665889489489
390
7.314839566634733
391
4.331202870692856
392
4.84576427968876
393
4.5030837026924315
394
4.543522472849018
395
4.504762568910493
396
5.027772044816395
397
2.4364665315156477
398
6.3814882998640705
399
2.679668634554702
400
6.668847958919661
401
7.912115932454656
402
3.1928437379115575
403
4.978918865989382
404
6.934768088023466
405
6.24601397120629
406
4.701505464500547
407
6.195494846003237
408
4.7492651984138075
409
4.001209238747663
410
5.998174152185552
411
4.620217480893602
412
6.530647639047578
413
4.636500499001301
414
8.364245232038153
415
6.992350101095493
416
4.921372273089634
417
3.1353762542531616
418
8.081365047567008
419
4.731183503866156
420
5.522992794916847
421
4.54344390164425
422
4.718445136750182
423
6.347566615499138
424
7.385497354133213
425
5.3805059289178665
426
5.090212945889578
427
3.868416561007793
428
4.430194574211589
429
6.577348575503839
430
6.444673634968649
431
5.

4.577867444702986
759
5.537748931764134
760
4.3590398242340145
761
3.773246361896585
762
3.545468200197039
763
4.616943552233225
764
3.462050356513579
765
4.700767784985418
766
7.942554841840363
767
6.837605132632195
768
6.312737777216634
769
4.928569345775916
770
4.1910656909373145
771
4.432311767731225
772
4.289342637100125
773
3.9706514287508368
774
2.5973499775074993
775
5.972751127091474
776
7.82640883851715
777
4.127829449008919
778
5.084141949644902
779
4.611597036716484
780
5.73141588936819
781
6.16176269182105
782
4.707066790678711
783
6.70121852355467
784
7.4538606767306845
785
4.6713383729522135
786
6.102699538989967
787
4.502980349425525
788
3.9239380496073144
789
4.528827212010791
790
6.103875521589811
791
4.502449279554881
792
4.621251439849596
793
5.527979969153742
794
6.861418837349806
795
4.912940014245834
796
6.658189973702212
797
4.8526000006981675
798
8.548222644235624
799
3.419048136991779
800
4.885725135445757
801
4.980211231643623
802
5.799270291348531
803
4.9779

4.6413701986930604
1127
4.904373079347788
1128
3.4228409731137637
1129
6.942604788890784
1130
7.265523161302593
1131
4.0818687668188005
1132
5.5107942515074875
1133
4.298454600320278
1134
5.846876676187532
1135
4.5987284972138625
1136
5.830220852028272
1137
5.351570514201896
1138
4.219689736574129
1139
8.300164467940483
1140
4.706820957681931
1141
5.327719728566642
1142
9.883916744933973
1143
4.426937872377119
1144
4.526090588792601
1145
6.045052710706308
1146
3.787012851574744
1147
3.6691640990607572
1148
5.613332488499744
1149
4.287746589926192
1150
4.678877243198421
1151
4.740249855428505
1152
8.401435049017145
1153
7.850934215281679
1154
4.607035070243103
1155
5.152242480104412
1156
3.3427208887551796
1157
4.815063723783693
1158
3.406928392671593
1159
6.247048800772277
1160
6.153661465045104
1161
4.4095350983789885
1162
6.761962293725981
1163
6.108685645539516
1164
4.969958281954323
1165
4.2827735372766345
1166
4.352088304545617
1167
3.9748450280265137
1168
4.5934078686270805
1169


4.8021482716409105
1486
4.249430801744007
1487
4.228355878884172
1488
5.497974582689515
1489
6.11563744319032
1490
3.896805800649899
1491
4.824383457412649
1492
4.803472357638648
1493
7.095681378513236
1494
4.258636331196273
1495
6.7044113360480955
1496
4.646741170499093
1497
4.687305625119469
1498
5.969334838260339
1499
7.691191911881276
1500
8.039780071142708
1501
4.430978818576202
1502
8.362742570327711
1503
4.893673068827459
1504
4.646223710620468
1505
5.403883192610433
1506
5.333508587271712
1507
4.4851377376787855
1508
3.718677210559278
1509
3.409826037993966
1510
4.534802691065812
1511
4.751616126526507
1512
4.688156278834564
1513
9.195274960770384
1514
5.351299531490499
1515
4.2967657604906035
1516
6.047930811281397
1517
4.514565569470625
1518
6.7951193555971825
1519
4.941883256320699
1520
5.391027755287099
1521
5.147189543831114
1522
3.865860299420581
1523
5.800705488561212
1524
4.197807584100245
1525
2.547557322352853
1526
6.741100550539516
1527
5.814614286153245
1528
5.63886

...

7.176465007823618
18631
4.9139809098665825
18632
3.676787374235629
18633
4.512714119107605
18634
4.808134587365466
18635
4.978519119080845
18636
5.008214373833605
18637
4.70487994174662
18638
4.856862079370792
18639
4.89806315866649
18640
4.803019552246283
18641
4.964402987863273
18642
3.304478062652082
18643
3.7263531447697087
18644
5.266557163903836
18645
6.483541605684332
18646
4.935334681595691
18647
5.106017985597189
18648
3.5728701535288145
18649
4.690417487366755
18650
4.748349283817584
18651
4.031759621630386
18652
4.814751683861774
18653
4.509921304897185
18654
4.615507464922431
18655
4.207020590619021
18656
4.14336467030532
18657
4.7204865997391074
18658
4.436295707372542
18659
4.65895656448397
18660
4.942813809815776
18661
4.173279911483419
18662
4.386798653907106
18663
4.427058174662857
18664
5.273887296380995
18665
3.7803266013283814
18666
4.19041056719667
18667
4.905756866349998
18668
4.733652760802354
18669
4.942641877590994
18670
3.8367406985724815
18671
4.9890929361288

4.84318785930652
18973
4.295584395799187
18974
5.65281426531918
18975
5.098255875233064
18976
7.028271537972861
18977
4.741387779267569
18978
4.421966819047732
18979
3.6496225228707218
18980
4.846344089292197
18981
4.815990566289436
18982
3.4282438232492725
18983
4.745744012873391
18984
4.721749434085263
18985
4.2088702162774165
18986
4.317520231490645
18987
4.776398357103146
18988
5.559066081367311
18989
4.774675096416593
18990
4.860004015121712
18991
4.14396289852146
18992
4.9170221132729806
18993
4.808815684982409
18994
4.082135059866797
18995
4.724020404157763
18996
5.337826823269056
18997
5.032306830871138
18998
4.5092145501475684
18999
4.558241169953771
19000
4.572763010584126
19001
5.163238440003273
19002
3.7490035690023147
19003
4.991181147685092
19004
4.4044934515906355
19005
4.669751421992358
19006
4.7023767296463
19007
5.10180408846438
19008
5.003007176966726
19009
4.015525184545483
19010
4.0902125898832296
19011
4.888258270193191
19012
4.926734949510216
19013
4.896215168462

4.629055014828587
19318
5.324656268860479
19319
4.200879066663145
19320
4.414120615715348
19321
4.655525656806854
19322
4.534958043499022
19323
4.685722446541954
19324
3.43216827594803
19325
4.554303123845411
19326
4.9567145942931035
19327
4.9050601877189495
19328
4.513372188222565
19329
3.714083257605685
19330
4.627935082769648
19331
4.591984377778602
19332
3.7966369915278646
19333
4.987623166433079
19334
4.837871103426941
19335
5.598363764974438
19336
6.560289001230738
19337
3.415064765293083
19338
4.933971571221285
19339
4.241605401342408
19340
5.095994236546319
19341
4.752736029694537
19342
4.094516423142409
19343
3.709594176125182
19344
3.935802417997002
19345
8.618402717825347
19346
4.391054624964063
19347
4.104432807387745
19348
4.689833223016403
19349
4.275911760085052
19350
2.961529142284597
19351
4.073250149209275
19352
4.229266228458316
19353
3.882644142636896
19354
4.184641033611216
19355
4.731278458869427
19356
4.690102249246941
19357
4.7786602068157045
19358
4.85096043633

4.3279023374152965
19662
4.971101651689294
19663
4.931120210685929
19664
6.319128590516824
19665
5.191075957771313
19666
4.978456372867529
19667
4.667191289427448
19668
4.7102789209152744
19669
4.792820729404122
19670
5.9958999466539815
19671
4.732273445300841
19672
4.868800465544432
19673
5.108913698547944
19674
4.480120265106795
19675
5.069828789945443
19676
3.4824387719705294
19677
4.143453143472982
19678
4.810234787459074
19679
5.5546284004102375
19680
4.524435561798658
19681
4.784198637242906
19682
3.689801128861844
19683
3.886479723774639
19684
4.250210440075955
19685
5.240467330261731
19686
4.9619903694573715
19687
4.877414235754424
19688
4.633801657787342
19689
4.6734598609898965
19690
3.428348620563037
19691
4.778415415858616
19692
4.909325915948737
19693
8.539419849617172
19694
4.75237212959608
19695
4.834568885885332
19696
4.1337225558585535
19697
4.424742159436233
19698
3.806082280895778
19699
5.488965542355333
19700
4.632678801688974
19701
4.130094757629493
19702
4.8271382

4.141856413984383
20007
4.628021394218073
20008
5.04942815740268
20009
5.008473145913456
20010
4.56175089296014
20011
4.053759607755627
20012
4.4635032797657415
20013
4.494929540078853
20014
7.9131985792631685
20015
4.6637269565959265
20016
4.718582261868553
20017
3.9694753551960926
20018
4.920835569785941
20019
4.623203363164093
20020
4.802098408242163
20021
4.859010565244211
20022
7.6523180671414135
20023
6.335059299672192
20024
4.93618376369688
20025
5.797720378974375
20026
6.787168334514991
20027
4.056768083965656
20028
4.4236340594538355
20029
3.4144481807291136
20030
5.160091421667159
20031
4.844413279742174
20032
5.099987832179879
20033
4.238488289198012
20034
4.803495064250163
20035
4.48790951621711
20036
4.9649365902244025
20037
5.812205635512054
20038
4.58773065992084
20039
4.848919278669665
20040
4.7582305453880975
20041
4.039993125042772
20042
3.9992813552744995
20043
8.586368202421868
20044
3.6104044919402014
20045
4.394184904886357
20046
4.681306368237706
20047
3.90539533

In [156]:
df['length']=length
df.head()

,isTrueTrack,pdg,completeness,purity,x,z,length
0,1,2212,1.000000,0.909091,"[1.72973107539, 1.64524385622, 1.51650142701, ...","[-1.69030850946, -1.60579308398, -1.5212776585...",4.785806
1,1,2212,0.733333,1.000000,"[-1.47290088225, -1.33730088739, -1.0958667502...","[-1.60356745147, -1.33630620956, -1.0690449676...",4.449725
2,0,11,0.892892,0.998911,"[1.6456069206, 1.63333808742, 1.61401467518, 1...","[-1.81425037168, -1.7974536461, -1.78065692051...",7.127999
3,1,2212,0.989362,0.965398,"[-1.36891816532, -1.37278190977, -1.3722299462...","[-1.59732439026, -1.57583288679, -1.5543413833...",4.540850
4,1,2212,0.875000,0.875000,"[-1.39488761287, -1.32920904814, -1.271740304,...","[-1.65144564769, -1.48630108292, -1.3211565181...",4.564701


In [68]:
pc2Var = pc2Var(df)


(41, 2)
(13, 2)
(247, 2)
(136, 2)
(21, 2)
(317, 2)
(37, 2)
(73, 2)
(242, 2)
(38, 2)
(57, 2)
(71, 2)
(8, 2)
(12, 2)
(164, 2)
(7, 2)
(5, 2)
(41, 2)
(41, 2)
(24, 2)
(281, 2)
(25, 2)
(46, 2)
(485, 2)
(59, 2)
(59, 2)
(116, 2)
(433, 2)
(235, 2)
(56, 2)
(72, 2)
(351, 2)
(63, 2)
(31, 2)
(29, 2)
(398, 2)
(5, 2)
(3, 2)
(171, 2)
(23, 2)
(116, 2)
(251, 2)
(6, 2)
(4, 2)
(1397, 2)
(54, 2)
(33, 2)
(9, 2)
(22, 2)
(73, 2)
(75, 2)
(53, 2)
(1436, 2)
(6, 2)
(580, 2)
(47, 2)
(338, 2)
(35, 2)
(141, 2)
(7, 2)
(169, 2)
(48, 2)
(120, 2)
(28, 2)
(48, 2)
(8, 2)
(860, 2)
(6, 2)
(26, 2)
(6, 2)
(112, 2)
(394, 2)
(9, 2)
(140, 2)
(219, 2)
(10, 2)
(96, 2)
(517, 2)
(16, 2)
(893, 2)
(8, 2)
(157, 2)
(45, 2)
(1593, 2)
(16, 2)
(26, 2)
(4, 2)
(217, 2)
(70, 2)
(58, 2)
(15, 2)
(28, 2)
(8, 2)
(12, 2)
(9, 2)
(645, 2)
(30, 2)
(716, 2)
(125, 2)
(19, 2)
(4, 2)
(7, 2)
(426, 2)
(20, 2)
(5, 2)
(142, 2)
(23, 2)
(9, 2)
(746, 2)
(223, 2)
(456, 2)
(392, 2)
(291, 2)
(21, 2)
(534, 2)
(99, 2)
(22, 2)
(693, 2)
(67, 2)
(16, 2)
(22, 2)
(5, 2)


(22, 2)
(13, 2)
(3, 2)
(263, 2)
(21, 2)
(31, 2)
(14, 2)
(41, 2)
(611, 2)
(9, 2)
(91, 2)
(8, 2)
(1527, 2)
(37, 2)
(30, 2)
(3, 2)
(3, 2)
(188, 2)
(30, 2)
(49, 2)
(519, 2)
(13, 2)
(4, 2)
(9, 2)
(6, 2)
(563, 2)
(184, 2)
(102, 2)
(28, 2)
(3, 2)
(64, 2)
(20, 2)
(60, 2)
(10, 2)
(894, 2)
(49, 2)
(396, 2)
(162, 2)
(177, 2)
(55, 2)
(140, 2)
(53, 2)
(185, 2)
(218, 2)
(264, 2)
(49, 2)
(24, 2)
(7, 2)
(1658, 2)
(28, 2)
(30, 2)
(122, 2)
(4, 2)
(33, 2)
(68, 2)
(275, 2)
(139, 2)
(118, 2)
(16, 2)
(461, 2)
(7, 2)
(4, 2)
(1265, 2)
(41, 2)
(153, 2)
(84, 2)
(184, 2)
(33, 2)
(50, 2)
(386, 2)
(187, 2)
(26, 2)
(503, 2)
(39, 2)
(7, 2)
(902, 2)
(19, 2)
(4, 2)
(7, 2)
(7, 2)
(206, 2)
(229, 2)
(69, 2)
(11, 2)
(12, 2)
(34, 2)
(16, 2)
(172, 2)
(73, 2)
(25, 2)
(226, 2)
(326, 2)
(87, 2)
(13, 2)
(152, 2)
(22, 2)
(204, 2)
(19, 2)
(81, 2)
(16, 2)
(13, 2)
(17, 2)
(549, 2)
(19, 2)
(921, 2)
(62, 2)
(49, 2)
(26, 2)
(342, 2)
(694, 2)
(258, 2)
(3, 2)
(306, 2)
(65, 2)
(372, 2)
(841, 2)
(73, 2)
(10, 2)
(18, 2)
(8, 2)
(831, 2)
(89

(46, 2)
(46, 2)
(446, 2)
(302, 2)
(142, 2)
(142, 2)
(9, 2)
(80, 2)
(137, 2)
(316, 2)
(127, 2)
(20, 2)
(637, 2)
(17, 2)
(4, 2)
(86, 2)
(276, 2)
(36, 2)
(863, 2)
(902, 2)
(92, 2)
(13, 2)
(12, 2)
(6, 2)
(43, 2)
(3, 2)
(3, 2)
(348, 2)
(109, 2)
(495, 2)
(44, 2)
(10, 2)
(19, 2)
(54, 2)
(24, 2)
(349, 2)
(7, 2)
(54, 2)
(48, 2)
(73, 2)
(32, 2)
(231, 2)
(70, 2)
(604, 2)
(60, 2)
(67, 2)
(29, 2)
(29, 2)
(9, 2)
(3, 2)
(12, 2)
(6, 2)
(497, 2)
(45, 2)
(61, 2)
(54, 2)
(113, 2)
(75, 2)
(211, 2)
(22, 2)
(52, 2)
(555, 2)
(21, 2)
(444, 2)
(42, 2)
(21, 2)
(36, 2)
(945, 2)
(13, 2)
(47, 2)
(5, 2)
(4, 2)
(26, 2)
(32, 2)
(5, 2)
(169, 2)
(93, 2)
(115, 2)
(98, 2)
(110, 2)
(411, 2)
(11, 2)
(1459, 2)
(196, 2)
(118, 2)
(387, 2)
(64, 2)
(580, 2)
(78, 2)
(371, 2)
(494, 2)
(19, 2)
(19, 2)
(52, 2)
(752, 2)
(8, 2)
(128, 2)
(67, 2)
(15, 2)
(1000, 2)
(208, 2)
(26, 2)
(581, 2)
(843, 2)
(41, 2)
(82, 2)
(141, 2)
(128, 2)
(4, 2)
(322, 2)
(436, 2)
(5, 2)
(36, 2)
(168, 2)
(255, 2)
(70, 2)
(370, 2)
(195, 2)
(41, 2)
(154, 2)
(66,

(393, 2)
(5, 2)
(22, 2)
(13, 2)
(166, 2)
(15, 2)
(631, 2)
(8, 2)
(245, 2)
(44, 2)
(219, 2)
(9, 2)
(208, 2)
(188, 2)
(28, 2)
(3, 2)
(1372, 2)
(18, 2)
(9, 2)
(11, 2)
(803, 2)
(12, 2)
(377, 2)
(44, 2)
(49, 2)
(927, 2)
(121, 2)
(1188, 2)
(27, 2)
(102, 2)
(35, 2)
(3, 2)
(350, 2)
(94, 2)
(3, 2)
(62, 2)
(412, 2)
(5, 2)
(490, 2)
(81, 2)
(3, 2)
(516, 2)
(43, 2)
(5, 2)
(46, 2)
(152, 2)
(19, 2)
(6, 2)
(133, 2)
(5, 2)
(32, 2)
(8, 2)
(1040, 2)
(111, 2)
(29, 2)
(8, 2)
(2559, 2)
(46, 2)
(718, 2)
(9, 2)
(97, 2)
(23, 2)
(78, 2)
(179, 2)
(101, 2)
(61, 2)
(9, 2)
(139, 2)
(370, 2)
(198, 2)
(411, 2)
(1369, 2)
(90, 2)
(369, 2)
(289, 2)
(33, 2)
(55, 2)
(1169, 2)
(768, 2)
(20, 2)
(271, 2)
(16, 2)
(5, 2)
(34, 2)
(116, 2)
(140, 2)
(89, 2)
(54, 2)
(3, 2)
(335, 2)
(28, 2)
(47, 2)
(114, 2)
(9, 2)
(1377, 2)
(71, 2)
(30, 2)
(154, 2)
(136, 2)
(21, 2)
(710, 2)
(664, 2)
(59, 2)
(33, 2)
(153, 2)
(207, 2)
(75, 2)
(58, 2)
(93, 2)
(35, 2)
(23, 2)
(98, 2)
(487, 2)
(127, 2)
(21, 2)
(774, 2)
(9, 2)
(38, 2)
(40, 2)
(104, 2)
(3

(6, 2)
(43, 2)
(61, 2)
(89, 2)
(11, 2)
(18, 2)
(53, 2)
(73, 2)
(224, 2)
(1402, 2)
(17, 2)
(1901, 2)
(96, 2)
(478, 2)
(76, 2)
(16, 2)
(4, 2)
(518, 2)
(76, 2)
(82, 2)
(691, 2)
(924, 2)
(6, 2)
(58, 2)
(94, 2)
(155, 2)
(152, 2)
(24, 2)
(114, 2)
(6, 2)
(54, 2)
(313, 2)
(531, 2)
(4, 2)
(90, 2)
(232, 2)
(289, 2)
(144, 2)
(46, 2)
(12, 2)
(5, 2)
(177, 2)
(28, 2)
(45, 2)
(155, 2)
(122, 2)
(198, 2)
(50, 2)
(194, 2)
(79, 2)
(49, 2)
(52, 2)
(29, 2)
(15, 2)
(692, 2)
(22, 2)
(122, 2)
(361, 2)
(182, 2)
(29, 2)
(4, 2)
(7, 2)
(4, 2)
(504, 2)
(107, 2)
(3, 2)
(1426, 2)
(146, 2)
(60, 2)
(22, 2)
(22, 2)
(597, 2)
(251, 2)
(29, 2)
(22, 2)
(81, 2)
(151, 2)
(112, 2)
(223, 2)
(16, 2)
(152, 2)
(13, 2)
(562, 2)
(32, 2)
(8, 2)
(14, 2)
(705, 2)
(82, 2)
(22, 2)
(457, 2)
(385, 2)
(12, 2)
(181, 2)
(1207, 2)
(20, 2)
(7, 2)
(184, 2)
(14, 2)
(237, 2)
(201, 2)
(4, 2)
(53, 2)
(11, 2)
(474, 2)
(74, 2)
(10, 2)
(14, 2)
(85, 2)
(143, 2)
(68, 2)
(28, 2)
(320, 2)
(7, 2)
(460, 2)
(20, 2)
(88, 2)
(25, 2)
(13, 2)
(10, 2)
(6, 2)
(103

...

(106, 2)
(6, 2)
(1210, 2)
(46, 2)
(412, 2)
(66, 2)
(236, 2)
(54, 2)
(29, 2)
(16, 2)
(1139, 2)
(18, 2)
(302, 2)
(927, 2)
(1844, 2)
(7, 2)
(76, 2)
(512, 2)
(93, 2)
(3, 2)
(75, 2)
(230, 2)
(43, 2)
(6, 2)
(338, 2)
(87, 2)
(118, 2)
(187, 2)
(408, 2)
(5, 2)
(277, 2)
(100, 2)
(49, 2)
(17, 2)
(194, 2)
(66, 2)
(40, 2)
(23, 2)
(326, 2)
(38, 2)
(5, 2)
(21, 2)
(16, 2)
(3, 2)
(53, 2)
(392, 2)
(12, 2)
(355, 2)
(9, 2)
(258, 2)
(778, 2)
(7, 2)
(16, 2)
(25, 2)
(86, 2)
(14, 2)
(6, 2)
(18, 2)
(756, 2)
(106, 2)
(283, 2)
(5, 2)
(92, 2)
(17, 2)
(686, 2)
(53, 2)
(1193, 2)
(10, 2)
(7, 2)
(216, 2)
(28, 2)
(102, 2)
(257, 2)
(25, 2)
(5, 2)
(3, 2)
(215, 2)
(30, 2)
(3, 2)
(357, 2)
(62, 2)
(278, 2)
(1575, 2)
(21, 2)
(300, 2)
(4, 2)
(18, 2)
(714, 2)
(210, 2)
(134, 2)
(12, 2)
(29, 2)
(70, 2)
(41, 2)
(269, 2)
(14, 2)
(10, 2)
(6, 2)
(129, 2)
(44, 2)
(3, 2)
(127, 2)
(3, 2)
(33, 2)
(593, 2)
(123, 2)
(719, 2)
(33, 2)
(203, 2)
(86, 2)
(113, 2)
(145, 2)
(130, 2)
(1682, 2)
(8, 2)
(37, 2)
(154, 2)
(33, 2)
(1028, 2)
(316, 2)
(

(69, 2)
(49, 2)
(40, 2)
(77, 2)
(285, 2)
(78, 2)
(69, 2)
(18, 2)
(149, 2)
(45, 2)
(19, 2)
(200, 2)
(193, 2)
(79, 2)
(823, 2)
(52, 2)
(332, 2)
(210, 2)
(7, 2)
(75, 2)
(65, 2)
(13, 2)
(19, 2)
(179, 2)
(5, 2)
(35, 2)
(107, 2)
(17, 2)
(1065, 2)
(6, 2)
(296, 2)
(42, 2)
(80, 2)
(82, 2)
(9, 2)
(207, 2)
(193, 2)
(17, 2)
(29, 2)
(1282, 2)
(47, 2)
(985, 2)
(3, 2)
(347, 2)
(984, 2)
(11, 2)
(1373, 2)
(3, 2)
(539, 2)
(78, 2)
(768, 2)
(62, 2)
(84, 2)
(107, 2)
(32, 2)
(2489, 2)
(12, 2)
(193, 2)
(138, 2)
(109, 2)
(223, 2)
(4, 2)
(1595, 2)
(395, 2)
(1060, 2)
(3, 2)
(76, 2)
(347, 2)
(68, 2)
(8, 2)
(50, 2)
(561, 2)
(527, 2)
(78, 2)
(288, 2)
(928, 2)
(195, 2)
(159, 2)
(90, 2)
(27, 2)
(65, 2)
(4, 2)
(206, 2)
(25, 2)
(675, 2)
(66, 2)
(14, 2)
(24, 2)
(949, 2)
(36, 2)
(1074, 2)
(1940, 2)
(149, 2)
(75, 2)
(41, 2)
(133, 2)
(1458, 2)
(8, 2)
(2512, 2)
(30, 2)
(7, 2)
(998, 2)
(9, 2)
(82, 2)
(52, 2)
(925, 2)
(49, 2)
(107, 2)
(41, 2)
(6, 2)
(12, 2)
(93, 2)
(6, 2)
(151, 2)
(387, 2)
(337, 2)
(219, 2)
(38, 2)
(66, 2)
(

(1286, 2)
(48, 2)
(16, 2)
(15, 2)
(3, 2)
(1846, 2)
(64, 2)
(23, 2)
(4, 2)
(84, 2)
(13, 2)
(15, 2)
(98, 2)
(135, 2)
(175, 2)
(349, 2)
(140, 2)
(52, 2)
(34, 2)
(193, 2)
(426, 2)
(320, 2)
(273, 2)
(643, 2)
(6, 2)
(456, 2)
(470, 2)
(14, 2)
(5, 2)
(16, 2)
(22, 2)
(258, 2)
(21, 2)
(3, 2)
(122, 2)
(17, 2)
(178, 2)
(160, 2)
(304, 2)
(6, 2)
(44, 2)
(186, 2)
(7, 2)
(8, 2)
(10, 2)
(465, 2)
(558, 2)
(107, 2)
(101, 2)
(16, 2)
(10, 2)
(16, 2)
(833, 2)
(907, 2)
(6, 2)
(82, 2)
(136, 2)
(286, 2)
(32, 2)
(724, 2)
(11, 2)
(18, 2)
(417, 2)
(31, 2)
(55, 2)
(85, 2)
(85, 2)
(61, 2)
(32, 2)
(74, 2)
(68, 2)
(7, 2)
(372, 2)
(4, 2)
(730, 2)
(35, 2)
(17, 2)
(225, 2)
(92, 2)
(5, 2)
(4, 2)
(1200, 2)
(12, 2)
(267, 2)
(95, 2)
(34, 2)
(18, 2)
(146, 2)
(23, 2)
(175, 2)
(71, 2)
(5, 2)
(277, 2)
(6, 2)
(10, 2)
(63, 2)
(137, 2)
(289, 2)
(164, 2)
(49, 2)
(565, 2)
(8, 2)
(42, 2)
(101, 2)
(9, 2)
(4, 2)
(64, 2)
(15, 2)
(76, 2)
(781, 2)
(307, 2)
(177, 2)
(52, 2)
(19, 2)
(136, 2)
(400, 2)
(157, 2)
(13, 2)
(449, 2)
(7, 2)
(15, 2)

(1000, 2)
(215, 2)
(64, 2)
(51, 2)
(412, 2)
(50, 2)
(586, 2)
(88, 2)
(68, 2)
(14, 2)
(4, 2)
(44, 2)
(4, 2)
(17, 2)
(6, 2)
(29, 2)
(2705, 2)
(162, 2)
(34, 2)
(39, 2)
(51, 2)
(67, 2)
(8, 2)
(826, 2)
(27, 2)
(4, 2)
(21, 2)
(12, 2)
(192, 2)
(13, 2)
(21, 2)
(22, 2)
(30, 2)
(256, 2)
(22, 2)
(37, 2)
(59, 2)
(13, 2)
(41, 2)
(233, 2)
(127, 2)
(29, 2)
(18, 2)
(14, 2)
(12, 2)
(6, 2)
(1198, 2)
(46, 2)
(84, 2)
(30, 2)
(85, 2)
(17, 2)
(23, 2)
(701, 2)
(110, 2)
(42, 2)
(188, 2)
(349, 2)
(224, 2)
(63, 2)
(25, 2)
(764, 2)
(18, 2)
(156, 2)
(448, 2)
(11, 2)
(272, 2)
(704, 2)
(141, 2)
(42, 2)
(224, 2)
(5, 2)
(4, 2)
(110, 2)
(39, 2)
(30, 2)
(15, 2)
(56, 2)
(423, 2)
(3, 2)
(13, 2)
(93, 2)
(44, 2)
(31, 2)
(40, 2)
(7, 2)
(7, 2)
(433, 2)
(13, 2)
(244, 2)
(62, 2)
(47, 2)
(175, 2)
(23, 2)
(12, 2)
(3, 2)
(29, 2)
(109, 2)
(168, 2)
(9, 2)
(21, 2)
(177, 2)
(13, 2)
(39, 2)
(442, 2)
(43, 2)
(205, 2)
(173, 2)
(91, 2)
(316, 2)
(60, 2)
(335, 2)
(114, 2)
(61, 2)
(52, 2)
(27, 2)
(337, 2)
(13, 2)
(70, 2)
(10, 2)
(68, 2)
(65

(193, 2)
(111, 2)
(3, 2)
(106, 2)
(24, 2)
(28, 2)
(342, 2)
(129, 2)
(356, 2)
(147, 2)
(65, 2)
(31, 2)
(176, 2)
(866, 2)
(29, 2)
(101, 2)
(11, 2)
(12, 2)
(228, 2)
(175, 2)
(29, 2)
(200, 2)
(7, 2)
(168, 2)
(367, 2)
(64, 2)
(1038, 2)
(460, 2)
(31, 2)
(610, 2)
(26, 2)
(3, 2)
(34, 2)
(44, 2)
(12, 2)
(134, 2)
(87, 2)
(20, 2)
(156, 2)
(47, 2)
(190, 2)
(1231, 2)
(7, 2)
(95, 2)
(416, 2)
(46, 2)
(9, 2)
(479, 2)
(30, 2)
(2881, 2)
(211, 2)
(597, 2)
(23, 2)
(3, 2)
(815, 2)
(960, 2)
(126, 2)
(53, 2)
(29, 2)
(32, 2)
(3, 2)
(68, 2)
(20, 2)
(266, 2)
(4, 2)
(875, 2)
(66, 2)
(584, 2)
(120, 2)
(205, 2)
(116, 2)
(83, 2)
(27, 2)
(289, 2)
(12, 2)
(638, 2)
(9, 2)
(466, 2)
(51, 2)
(454, 2)
(9, 2)
(7, 2)
(28, 2)
(971, 2)
(46, 2)
(10, 2)
(83, 2)
(49, 2)
(355, 2)
(9, 2)
(37, 2)
(4, 2)
(251, 2)
(265, 2)
(224, 2)
(87, 2)
(74, 2)
(422, 2)
(7, 2)
(819, 2)
(13, 2)
(22, 2)
(216, 2)
(156, 2)
(95, 2)
(66, 2)
(24, 2)
(1018, 2)
(96, 2)
(474, 2)
(8, 2)
(668, 2)
(273, 2)
(20, 2)
(9, 2)
(69, 2)
(23, 2)
(32, 2)
(149, 2)
(63, 2

In [69]:
pc2Var = [pc2Var[i][0] for i in range(len(pc2Var))]

In [71]:
def findEpsBinSearch(df):
    epsList=[]
    for i in range(len(df)):
        print(i)
        epses=[]
        eps = 0.001
        epsRange=np.arange(0,20, 0.001)
        l = 0
        h= len(epsRange)-1
        #clustering = DBSCAN(eps=epsRange, min_samples=2).fit(np.array(df['x'][i]).reshape(-1,1), df['z'][i])
        while l<=h:
            m= int((l+h)/2)
            clustering = DBSCAN(eps=epsRange[m], min_samples=2).fit(np.array(df['x'][i]).reshape(-1,1), df['z'][i])
            if (list(clustering.labels_).count(0)/len(clustering.labels_)< 1): 
                l = m + 1
            elif (list(clustering.labels_).count(0)/len(clustering.labels_) == 1): 
                h = m - 1
        
        epsList.append(epsRange[h])
    return epsList

In [153]:
len(pc2Var)

20243

In [33]:
def findEps(df):
    epsList = []
    for i in range(len(df)):
        print(i)
        epses=[]
        eps = 0.001
        clustering = DBSCAN(eps=eps, min_samples=2).fit(np.array(df['x'][i]).reshape(-1,1), df['z'][i])
        while (list(clustering.labels_).count(0)/len(clustering.labels_)< 1):
            eps+=0.001
            if eps > 20:
                eps += 1
            if eps> 100:
                break

            clustering = DBSCAN(eps=eps, min_samples=2).fit(np.array(df['x'][i]).reshape(-1,1), df['z'][i])

        epsList.append(eps)
    #print(eps)
    return epsList

In [72]:
epsList = findEpsBinSearch(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068


3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707


5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357


6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999


...

14186
14187
14188
14189
14190
14191
14192
14193
14194
14195
14196
14197
14198
14199
14200
14201
14202
14203
14204
14205
14206
14207
14208
14209
14210
14211
14212
14213
14214
14215
14216
14217
14218
14219
14220
14221
14222
14223
14224
14225
14226
14227
14228
14229
14230
14231
14232
14233
14234
14235
14236
14237
14238
14239
14240
14241
14242
14243
14244
14245
14246
14247
14248
14249
14250
14251
14252
14253
14254
14255
14256
14257
14258
14259
14260
14261
14262
14263
14264
14265
14266
14267
14268
14269
14270
14271
14272
14273
14274
14275
14276
14277
14278
14279
14280
14281
14282
14283
14284
14285
14286
14287
14288
14289
14290
14291
14292
14293
14294
14295
14296
14297
14298
14299
14300
14301
14302
14303
14304
14305
14306
14307
14308
14309
14310
14311
14312
14313
14314
14315
14316
14317
14318
14319
14320
14321
14322
14323
14324
14325
14326
14327
14328
14329
14330
14331
14332
14333
14334
14335
14336
14337
14338
14339
14340
14341
14342
14343
14344
14345
14346
14347
14348
14349
14350
14351
1435

15559
15560
15561
15562
15563
15564
15565
15566
15567
15568
15569
15570
15571
15572
15573
15574
15575
15576
15577
15578
15579
15580
15581
15582
15583
15584
15585
15586
15587
15588
15589
15590
15591
15592
15593
15594
15595
15596
15597
15598
15599
15600
15601
15602
15603
15604
15605
15606
15607
15608
15609
15610
15611
15612
15613
15614
15615
15616
15617
15618
15619
15620
15621
15622
15623
15624
15625
15626
15627
15628
15629
15630
15631
15632
15633
15634
15635
15636
15637
15638
15639
15640
15641
15642
15643
15644
15645
15646
15647
15648
15649
15650
15651
15652
15653
15654
15655
15656
15657
15658
15659
15660
15661
15662
15663
15664
15665
15666
15667
15668
15669
15670
15671
15672
15673
15674
15675
15676
15677
15678
15679
15680
15681
15682
15683
15684
15685
15686
15687
15688
15689
15690
15691
15692
15693
15694
15695
15696
15697
15698
15699
15700
15701
15702
15703
15704
15705
15706
15707
15708
15709
15710
15711
15712
15713
15714
15715
15716
15717
15718
15719
15720
15721
15722
15723
15724
1572

16937
16938
16939
16940
16941
16942
16943
16944
16945
16946
16947
16948
16949
16950
16951
16952
16953
16954
16955
16956
16957
16958
16959
16960
16961
16962
16963
16964
16965
16966
16967
16968
16969
16970
16971
16972
16973
16974
16975
16976
16977
16978
16979
16980
16981
16982
16983
16984
16985
16986
16987
16988
16989
16990
16991
16992
16993
16994
16995
16996
16997
16998
16999
17000
17001
17002
17003
17004
17005
17006
17007
17008
17009
17010
17011
17012
17013
17014
17015
17016
17017
17018
17019
17020
17021
17022
17023
17024
17025
17026
17027
17028
17029
17030
17031
17032
17033
17034
17035
17036
17037
17038
17039
17040
17041
17042
17043
17044
17045
17046
17047
17048
17049
17050
17051
17052
17053
17054
17055
17056
17057
17058
17059
17060
17061
17062
17063
17064
17065
17066
17067
17068
17069
17070
17071
17072
17073
17074
17075
17076
17077
17078
17079
17080
17081
17082
17083
17084
17085
17086
17087
17088
17089
17090
17091
17092
17093
17094
17095
17096
17097
17098
17099
17100
17101
17102
1710

18303
18304
18305
18306
18307
18308
18309
18310
18311
18312
18313
18314
18315
18316
18317
18318
18319
18320
18321
18322
18323
18324
18325
18326
18327
18328
18329
18330
18331
18332
18333
18334
18335
18336
18337
18338
18339
18340
18341
18342
18343
18344
18345
18346
18347
18348
18349
18350
18351
18352
18353
18354
18355
18356
18357
18358
18359
18360
18361
18362
18363
18364
18365
18366
18367
18368
18369
18370
18371
18372
18373
18374
18375
18376
18377
18378
18379
18380
18381
18382
18383
18384
18385
18386
18387
18388
18389
18390
18391
18392
18393
18394
18395
18396
18397
18398
18399
18400
18401
18402
18403
18404
18405
18406
18407
18408
18409
18410
18411
18412
18413
18414
18415
18416
18417
18418
18419
18420
18421
18422
18423
18424
18425
18426
18427
18428
18429
18430
18431
18432
18433
18434
18435
18436
18437
18438
18439
18440
18441
18442
18443
18444
18445
18446
18447
18448
18449
18450
18451
18452
18453
18454
18455
18456
18457
18458
18459
18460
18461
18462
18463
18464
18465
18466
18467
18468
1846

19679
19680
19681
19682
19683
19684
19685
19686
19687
19688
19689
19690
19691
19692
19693
19694
19695
19696
19697
19698
19699
19700
19701
19702
19703
19704
19705
19706
19707
19708
19709
19710
19711
19712
19713
19714
19715
19716
19717
19718
19719
19720
19721
19722
19723
19724
19725
19726
19727
19728
19729
19730
19731
19732
19733
19734
19735
19736
19737
19738
19739
19740
19741
19742
19743
19744
19745
19746
19747
19748
19749
19750
19751
19752
19753
19754
19755
19756
19757
19758
19759
19760
19761
19762
19763
19764
19765
19766
19767
19768
19769
19770
19771
19772
19773
19774
19775
19776
19777
19778
19779
19780
19781
19782
19783
19784
19785
19786
19787
19788
19789
19790
19791
19792
19793
19794
19795
19796
19797
19798
19799
19800
19801
19802
19803
19804
19805
19806
19807
19808
19809
19810
19811
19812
19813
19814
19815
19816
19817
19818
19819
19820
19821
19822
19823
19824
19825
19826
19827
19828
19829
19830
19831
19832
19833
19834
19835
19836
19837
19838
19839
19840
19841
19842
19843
19844
1984

In [73]:
RSS = RSS(df)
RSS2 = RSS2(df)

In [18]:
epsList


[0.2,
 0.5,
 4.899999999999999,
 0.7,
 0.1,
 4.799999999999999,
 0.6,
 14.999999999999963,
 1.8000000000000005,
 0.7,
 12.79999999999997,
 21.000000000000014,
 0.4,
 0.2,
 18.9,
 0.1,
 0.30000000000000004,
 0.2,
 6.499999999999993,
 0.1,
 1.6000000000000003,
 0.30000000000000004,
 0.8999999999999999,
 9.299999999999983,
 0.1,
 5.699999999999996,
 22.100000000000016,
 14.799999999999963,
 0.8999999999999999,
 2.400000000000001,
 1.3,
 0.5,
 0.7,
 1.6000000000000003,
 0.9999999999999999,
 7.099999999999991,
 0.2,
 0.5,
 13.09999999999997,
 0.7999999999999999,
 3.1000000000000014,
 1.4000000000000001,
 0.4,
 0.1,
 3.600000000000002,
 2.2000000000000006,
 0.2,
 0.6,
 0.8999999999999999,
 9.399999999999983,
 0.1,
 0.4,
 22.100000000000016,
 0.1,
 10.499999999999979,
 1.3,
 8.799999999999985,
 0.4,
 1.0999999999999999,
 0.1,
 1.7000000000000004,
 0.9999999999999999,
 1.0999999999999999,
 0.1,
 0.30000000000000004,
 0.30000000000000004,
 5.799999999999995,
 4.699999999999999,
 0.6,
 7.9999999

In [74]:
nHits = [len(df.x[i]) for i in range(len(df)) ]

In [201]:
def slidingWindowHelper(x,y, n, length):
    
    reg=LinearRegression().fit(x.reshape(-1,1), y)
    reg.coef_, reg.intercept_
    d=length/50
    m = reg.coef_[0]
    
    
    if m == 0:
        return 0

    dx= d/(m*m +1)**0.5 
    
    def y_pred(x):
        return float(m*x+reg.intercept_)
    
    intercepts = [(1/float(m))*i + y_pred(i) for i in np.arange(min(x),max(x),float(dx)) ]
    #print(len(intercepts), m)
    perp_dist=0
    #plt.scatter(x, y)
    #plt.show()

    for i in range(0,len(intercepts)-1):
        bighits=[]
        smallhits=[]
        
        if m > 0:
            for j in range(len(x)):
                if y[j]<-1/m*x[j] + intercepts[min(i+n+1,len(intercepts)-1)] and y[j]>-1/m*x[j] + intercepts[max(i-n,0)]:
                    bighits.append(j)
                if y[j]<-1/m*x[j] + intercepts[min(i+1,len(intercepts)-1)] and y[j]>-1/m*x[j] + intercepts[i]:
                    smallhits.append(j)
        else:
            for j in range(len(x)):
                if y[j]>-1/m*x[j] + intercepts[min(i+n+1,len(intercepts)-1)] and y[j]<-1/m*x[j] + intercepts[max(i-n,0)]:
                    bighits.append(j)
                if y[j]>-1/m*x[j] + intercepts[min(i+1,len(intercepts)-1)] and y[j]<-1/m*x[j] + intercepts[i]:
                    smallhits.append(j)
            
        #print(smallhits)
        if len(smallhits)==0:
            continue
        new_x = np.array([x[k] for k in bighits])
        new_y = [y[k] for k in bighits]
        
        new_reg=LinearRegression().fit(new_x.reshape(-1,1), new_y)
        for l in smallhits:
            perp_dist+=abs(-new_reg.coef_[0]*x[l] + y[l] -new_reg.intercept_)/(new_reg.coef_[0]**2 + 1)**0.5
    return perp_dist/len(x)      

In [202]:
def SlidingWindow(df,n):
    slidingWindowList=[]
    for i in range(len(df)):
        
        #print(df.x[i], df.z[i])
        answer = slidingWindowHelper(np.array(df.x[i]),np.array(df.z[i]), n, df.length[i])
        print(i,answer)
        slidingWindowList.append(answer)
    return slidingWindowList

In [196]:
sliding = SlidingWindow(df,4)


0 0.00552460848831
1 0.00546416982082
2 0.111450497603
3 0.00474074712098
4 0.00494996210795
5 0.176043374339
6 0.111843708112
7 0.109954681202
8 0.00309150979218
9 0.00599439274096
10 0.0693495323825
11 0.0130020428119
12 0.125686110404
13 0.00723263438935
14 0.057314223164
15 4.95298696526e-17
16 0.408714843416
17 0.0295825215178
18 0.0195083057124
19 0.483180704155
20 0.00411146583908
21 0.0146791490286
22 0.147102966317
23 0.0789797773573
24 0.0044103257756
25 0.002987427708
26 0.00553507601914
27 0.335505088838
28 0.132865246879
29 0.132912824042
30 0.00210341456683
31 0.00293077510788
32 0.00749816774992
33 0.00885833655593
34 0.0242014567828
35 0.119622576473
36 0.0302081423128
37 0.0
38 0.0573548656849
39 0.00822295508146
40 0.130150660926
41 0.0801358877094
42 0.0
43 0.261063873381
44 0.139437529484
45 0.197347363803
46 0.148129294809
47 0.00316326680271
48 0.00315230860268
49 0.0019425606265
50 0.0197438554037
51 0.019807807643
52 0.127885727969
53 0.0
54 0.0966991034175
55 0

442 0.0639343589036
443 0.0408962147287
444 0.0952834292892
445 0.00335717565081
446 0.0137981056161
447 0.187833537959
448 0.00383441822786
449 0.0872809450461
450 0.109287154989
451 0.000800861710185
452 0.112411220441
453 0.00311244291175
454 0.146916092899
455 0.0
456 0.0585094909973
457 1.95446506744e-18
458 0.041319335997
459 0.111237922037
460 0.00126839736296
461 0.00607506764625
462 0.00241967649606
463 0.0516908622802
464 0.0048871086305
465 0.00373472926434
466 0.0328249653354
467 2.38699211263e-17
468 0.099201963125
469 0.00279046505744
470 0.0373389130535
471 0.104558277681
472 0.00976411829307
473 0.00169190066822
474 0.0526380799507
475 0.0654057541488
476 0.100966652767
477 0.0
478 0.127155946596
479 0.125215756584
480 0.343377276143
481 0.0
482 0.0155544503136
483 0.048405349075
484 0.0885200770826
485 0.00253097114599
486 0.134376612816
487 0.104744107453
488 0.0338648999508
489 0.00200259869132
490 0.0528738830057
491 0.0
492 0.0452944399488
493 0.105175962318
494 0.

872 0.0321913959693
873 0.0
874 0.0676946300976
875 0.0065059010473
876 0.00380309206321
877 0.0
878 0.0336270541235
879 7.13888693711e-17
880 0.0474051736297
881 0.00554292475188
882 0.00294457086933
883 0.107029266474
884 0.00379025074782
885 0.0362636544102
886 0.00213019963974
887 0.00166268940379
888 0.0200343281146
889 0.105018774133
890 0.00385165250256
891 0.0713177510577
892 0.249941574654
893 0.110152379094
894 0.0734142526719
895 0.0461077924007
896 0.00422700872702
897 0.227679324158
898 0.143561542363
899 0.0407672551025
900 0.0642947576492
901 0.0413102638632
902 0.0064029912146
903 0.110053747148
904 0.0151469810106
905 0.0681938101831
906 0.0412107548248
907 0.0106570124972
908 0.0569733087993
909 0.0271064788807
910 0.00210766821117
911 0.120555755369
912 0.107143027511
913 0.0199053869815
914 0.130514635741
915 0.094432752705
916 0.0535021861917
917 0.0379421511926
918 0.0963718363247
919 0.00215099476429
920 0.0603694378912
921 0.00180810743161
922 0.0976032071969
92

1280 0.278117559505
1281 0.00167764399717
1282 0.262616004691
1283 0.126051166785
1284 0.0900217295626
1285 0.00563469481782
1286 0.0260933126226
1287 0.0630455176632
1288 0.26967061098
1289 0.117996326418
1290 0.0585649315208
1291 0.0685863229454
1292 0.00332626723411
1293 0.124652895773
1294 0.230605423039
1295 0.0209554935101
1296 0.0525913202375
1297 0.0625819588333
1298 0.0973791517559
1299 0.0519651987426
1300 0.0504252008769
1301 0.0
1302 0.00266621027337
1303 0.00219296716672
1304 0.0139778804469
1305 0.129092142399
1306 0.133603027515
1307 0.100474334327
1308 0.0726984916923
1309 0.0017814194139
1310 0.00371008795254
1311 0.139527825713
1312 0.00278550289422
1313 0.0119447423241
1314 0.118642934051
1315 0.06438934811
1316 0.0417373344643
1317 0.0844214277434
1318 0.0830877497307
1319 0.11637184887
1320 0.0048690488122
1321 0.0892733158103
1322 0.00149413925551
1323 0.0994112334107
1324 0.00150563337756
1325 0.00412764332519
1326 0.110889143419
1327 0.0564583177192
1328 0.11014

1688 0.107722739064
1689 0.0974050639464
1690 0.0643491404313
1691 0.0506528298102
1692 0.00226447451922
1693 0.111461037714
1694 0.0810533563154
1695 1.00386953369e-16
1696 0.0114243875944
1697 0.252460088963
1698 0.0562848066169
1699 0.0399677902757
1700 1.06996517608e-17
1701 0.0
1702 0.128030671509
1703 0.143981566501
1704 0.0
1705 0.00744706060535
1706 0.12260603306
1707 0.0876950927311
1708 0.0187482210536
1709 0.119132521107
1710 0.00485610525611
1711 0.442597713192
1712 0.105001276893
1713 0.00725325643239
1714 0.141198310175
1715 0.0445542664594
1716 0.00833484586541
1717 0.099170840917
1718 0.00272514470811
1719 0.0522883466296
1720 0.0964051518317
1721 0.00459953127625
1722 0.161734612113
1723 0.129490781688
1724 0.11093101044
1725 0.034577670836
1726 0.0149531844748
1727 0.0214752033121
1728 0.0114350145894
1729 0.00152523937147
1730 0.116220570744
1731 0.128228919066
1732 0.00217976790302
1733 0.00296204730701
1734 0.123585214448
1735 0.12287442
1736 0.032833759509
1737 0.

...

18458 0.0404173014662
18459 0.149833499247
18460 0.00197695717626
18461 0.0021600759027
18462 0.00286338468505
18463 0.00426109834284
18464 0.00350333012876
18465 0.064253793664
18466 0.0981790429046
18467 0.00253673575428
18468 0.0433938314832
18469 0.0975938075842
18470 0.00855468289041
18471 0.0143078842656
18472 0.0126561931917
18473 0.0120130184504
18474 0.00352158345962
18475 0.0
18476 0.00104716650721
18477 0.00584106101998
18478 0.0
18479 3.02046262764e-17
18480 0.00824317364219
18481 0.00336359901378
18482 0.00895929135196
18483 0.12389506578
18484 0.116581890929
18485 0.0
18486 0.0299841734169
18487 0.0391678729082
18488 0.00417635784433
18489 0.00170855338804
18490 0.00392800203362
18491 0.00195759162529
18492 0.00380430773518
18493 0.00455449165501
18494 0.00971598414923
18495 0.0739915547973
18496 0.00344841906437
18497 0.00750651887302
18498 0.00864843076503
18499 0.666665218853
18500 0.0294301007706
18501 0.0148804747242
18502 0.201427391545
18503 0.271940259048
18504 0.

18842 0.00539651841297
18843 0.060260124211
18844 0.00622919918714
18845 0.00245361204793
18846 0.00454122853581
18847 0.0
18848 0.00754089287769
18849 0.0416014874723
18850 0.00610673474415
18851 0.0
18852 0.0265916769422
18853 0.00166261288731
18854 0.0124743926367
18855 0.00773654793703
18856 0.0123150021088
18857 0.103367095252
18858 0.0866274371749
18859 0.00736208392857
18860 0.0059287679273
18861 0.0174165835201
18862 0.00173521092416
18863 0.0167051553873
18864 0.00529255324489
18865 0.148593894154
18866 0.052368628071
18867 0.00498672934544
18868 0.0026342339447
18869 0.566623134302
18870 0.0103038614879
18871 0.00942969647768
18872 0.00533736789423
18873 0.0828090089677
18874 0.000231923952526
18875 0.122831171273
18876 0.00286596062111
18877 0.057623627281
18878 0.0149700229889
18879 0.22110582374
18880 2.06772359544e-17
18881 0.0
18882 0.00464156512545
18883 0.0265478826105
18884 0.00399379414367
18885 0.248146419614
18886 0.0622170013199
18887 0.00509196330107
18888 0.0189

19223 0.00127889643833
19224 1.19958620451e-17
19225 0.0
19226 0.0392018360015
19227 6.77460626508e-17
19228 0.00456189659688
19229 0.00251852272117
19230 0.00257285025224
19231 0.00462374290156
19232 0.00620219790002
19233 0.00497740005568
19234 0.0137929729862
19235 0.0182515341969
19236 0.054714047946
19237 0.0915968907658
19238 0.00914195350432
19239 0.00493032137679
19240 0.0975832770079
19241 0.00119736270774
19242 0.0
19243 0.0
19244 0.0716535406252
19245 0.00690665219046
19246 0.00775875741796
19247 6.039982336e-17
19248 0.125991104441
19249 0.0155536352986
19250 0.0
19251 0.00313013520752
19252 0.0104389071308
19253 0.00649816552747
19254 0.0467336720136
19255 0.0117763154334
19256 0.0117978119239
19257 0.0982820636136
19258 0.0486970001732
19259 0.00343310423756
19260 0.03340149138
19261 0.0126904536343
19262 0.003343004995
19263 0.00576678355081
19264 0.00459297693774
19265 1.3647251962e-17
19266 0.000976013488702
19267 0.0021664647085
19268 0.00676482213227
19269 0.00697603

19606 0.125159066082
19607 0.0412798855506
19608 0.0284285274648
19609 1.36545934503e-17
19610 0.00182527658577
19611 0.0032532998701
19612 0.0
19613 0.00750366432077
19614 0.00180813737122
19615 0.139414552374
19616 0.156163169103
19617 0.0
19618 0.00710207621979
19619 0.0589004765957
19620 0.0516521076381
19621 0.0430151981307
19622 0.00473605350512
19623 0.0273327534989
19624 0.223125206038
19625 0.00333374706136
19626 0.0
19627 0.00312677219077
19628 0.0024940183953
19629 0.069754202161
19630 0.00540530853076
19631 0.0560410241877
19632 0.0751160205786
19633 0.0256954085367
19634 0.00514715310233
19635 0.0
19636 0.103644612997
19637 0.0157819086494
19638 0.0267435439465
19639 0.00414908226562
19640 0.00674461670364
19641 2.84180680656e-17
19642 0.0715052680993
19643 0.00381406678987
19644 0.00950490188155
19645 0.0625238732745
19646 0.00323927572647
19647 0.0231901251084
19648 0.00278666637998
19649 1.10454636472e-17
19650 0.0153439941648
19651 0.00353799382782
19652 0.002207062154

19983 0.00172763339807
19984 0.00200436645605
19985 0.0248774983754
19986 0.00188710779537
19987 0.00229818753097
19988 0.00232071029438
19989 0.0868488715271
19990 0.0
19991 0.15351338368
19992 0.00429723893683
19993 0.0146008577023
19994 3.00739972245e-17
19995 0.0135806945645
19996 0.0815973270278
19997 0.0743146075732
19998 0.00403159055798
19999 0.00267612172855
20000 0.00335153676744
20001 0.0
20002 0.00242582421058
20003 0.0
20004 0.00397115367591
20005 0.00166045736622
20006 0.150845600975
20007 0.00553031902829
20008 0.00364642451674
20009 0.0873209768924
20010 0.0310267960278
20011 0.0
20012 0.00439122129274
20013 0.00192691813567
20014 0.11721249886
20015 0.00619079621026
20016 0.00317101653038
20017 0.0653281141583
20018 0.00407321105601
20019 0.00435351473299
20020 0.0174615442881
20021 0.0114361689324
20022 0.292998775219
20023 0.0981235265447
20024 0.00586778716891
20025 0.0667130021936
20026 0.0157471522407
20027 0.0589800210671
20028 0.0021784888532
20029 0.02853307181

sl

,isTrueTrack,pdg,completeness,purity,x,z,eps,pc2Var,nHits,RSS,RSS2
0,1,2212,1.000000,0.909091,"[119.384, 119.3, 119.172, 119.087, 119.007, 11...","[846.25, 846.55, 846.85, 847.15, 847.45, 847.7...",0.2,0.000057,41,0.020997,0.007951
1,1,2212,0.733333,1.000000,"[115.512, 115.676, 115.968, 116.284, 116.596, ...","[842.35, 842.65, 842.95, 843.25, 843.55, 843.8...",0.5,0.000663,13,0.032420,0.002150
2,0,11,0.892892,0.998911,"[165.12, 165.08, 165.017, 164.967, 164.921, 16...","[640.75, 641.05, 641.35, 641.65, 641.95, 642.2...",4.9,0.000467,247,1.072781,47.833154
3,1,2212,0.989362,0.965398,"[165.52, 165.513, 165.514, 165.499, 165.503, 1...","[638.35, 638.65, 638.95, 639.25, 639.55, 639.8...",0.7,0.000035,136,0.130221,0.216665
4,1,2212,0.875000,0.875000,"[164.341, 164.365, 164.386, 164.427, 164.472, ...","[632.05, 632.35, 632.65, 632.95, 633.25, 633.5...",0.1,0.001649,21,0.028268,0.008956


In [197]:
df['sliding_modified']=sliding

In [222]:
df['envelope']=area

In [164]:
df['eps'] =epsList
df['pc2Var'] = pc2Var
df['nHits']=nHits
df['RSS']=RSS
df['RSS2']=RSS2

In [223]:
df.head(10)

,isTrueTrack,pdg,completeness,purity,x,z,length,sliding,eps,pc2Var,nHits,RSS,RSS2,sliding_modified,envelope
0,1,2212,1.000000,0.909091,"[1.72973107539, 1.64524385622, 1.51650142701, ...","[-1.69030850946, -1.60579308398, -1.5212776585...",4.785806,0.005515,0.133,0.000208,41,0.015505,0.000631,0.005525,9.579459
1,1,2212,0.733333,1.000000,"[-1.47290088225, -1.33730088739, -1.0958667502...","[-1.60356745147, -1.33630620956, -1.0690449676...",4.449725,0.001786,0.365,0.000672,13,0.027864,0.001706,0.005464,8.916642
2,0,11,0.892892,0.998911,"[1.6456069206, 1.63333808742, 1.61401467518, 1...","[-1.81425037168, -1.7974536461, -1.78065692051...",7.127999,0.157412,1.486,0.039021,247,0.227368,0.149946,0.111450,15.046738
3,1,2212,0.989362,0.965398,"[-1.36891816532, -1.37278190977, -1.3722299462...","[-1.59732439026, -1.57583288679, -1.5543413833...",4.540850,0.004279,0.385,0.001711,136,0.051166,0.001112,0.004741,9.154247
4,1,2212,0.875000,0.875000,"[-1.39488761287, -1.32920904814, -1.271740304,...","[-1.65144564769, -1.48630108292, -1.3211565181...",4.564701,0.003981,0.229,0.002311,21,0.055676,0.002714,0.004950,9.176259
5,0,11,0.643982,1.000000,"[0.730457541548, 0.722967173309, 0.71040979126...","[-1.17115079142, -1.16100250394, -1.1508542164...",4.864244,0.164177,1.056,0.350141,317,0.715791,0.899496,0.176043,15.307257
6,0,11,0.052306,1.000000,"[-0.27650562668, -0.285789390916, -0.140343751...","[-1.68585446085, -1.59219587969, -1.4985372985...",3.347594,0.060274,0.614,0.408440,37,0.814280,0.892134,0.111844,10.396433
7,0,11,0.975904,0.852632,"[2.71422316111, 2.66330894845, 2.64608213648, ...","[-5.60633080866, -3.45458449408, -3.0242352311...",7.973702,0.131938,1.313,0.187135,73,0.416976,0.600453,0.109955,18.236148
8,1,2212,0.993789,0.977597,"[1.86185982421, 1.85832947259, 1.83323875929, ...","[-1.70506474519, -1.69168735279, -1.6783099604...",5.047310,0.003124,0.216,0.000390,242,0.017907,0.001061,0.003092,10.112642
9,0,11,0.960212,1.000000,"[-1.65000420287, -1.5678862995, -1.47599799403...","[-1.68705478457, -1.59586263406, -1.5046704835...",4.767060,0.006063,0.143,0.000255,38,0.018221,0.000262,0.005994,9.539038


In [224]:
df.to_csv('../BNB_nue data/big_data_nue_and_mu_scaledVars.csv')

In [204]:
len(pc2Var)

20243

TypeError: 'NoneType' object does not support item assignment

In [203]:
len(epsList)

20243

20232